In [36]:
import requests

In [37]:
endpoint = "http://localhost:7071/graphql"
prod_endpoint = ""

In [38]:
public_query = """
query Infobase {
  Infobase {
    Date
    Location
    region
    measureid
    fractionid
    viral_load
    seven_day_rolling_avg
    pruid
  }
  InfobaseTrend {
    fractionid
    region
    Location
    measureid
    LatestTrend
    pruid
  }
  partLUs {
    partID
    partType
    partLabel
    partDesc
    partInstr
    domainID
    specSet
    compSet
    groupID
    classID
    nomenclatureID
    ontologyRef
    latExp
    catSetID
    unitSetID
    aggScaleID
    aggSetID
    qualitySetID
    status
    firstRel
    lastUp
    changes
    methodSteps
    methodStepsRequired
    methodStepsOrder
    methodSets
    methodSetsRequired
    methodSetsOrder
    measures
    measuresRequired
    measuresOrder
    measureSets
    measureSetsRequired
    measureSetsOrder
    datasets
    datasetsRequired
    datasetsOrder
    sites
    sitesRequired
    sitesOrder
    samples
    samplesRequired
    samplesOrder
    addresses
    addressesRequired
    addressesOrder
    contacts
    contactsRequired
    contactsOrder
    organizations
    orgganizationsRequired
    organizationsOrder
    instruments
    instrumentsRequired
    instrumentsOrder
    polygons
    polygonsRequired
    polygonsOrder
    languageLUs
    translationLUs
    partLUs
    setLUs
    refLink
    dataType
    minValue
    maxValue
    minLength
    maxLength
  }
  setLUs {
    setID
    setType
    setValue
    partLabel
    notes
    partInstr
    status
    firstRel
    lastUp
    changes
  }
  languageLUs {
    langID
    langFam
    langName
    natName
    ISO6391
    ISO6392T
    ISO6392B
    ISO6393
    ISO6396
    notes
  }
}
"""



In [39]:
def contains_data(value):
    return isinstance(value, list) and len(value) > 0

def test_graphql_public_tables():
    response = requests.post(
        "http://localhost:7071/graphql", json={"query": public_query}
    )
    data = response.json()["data"]

    for key, value in data.items():
        assert contains_data(value), f"'{key}' is an empty."

def test_graphql_filter():
    filter_value = "Toronto"
    query =f"""
    query Infobase {{
    Infobase(filter: {{Location: {{contains: "{filter_value}"}}}}) {{
        Location
    }}
    }}
    """
    response = requests.post(
        "http://localhost:7071/graphql", json={"query": query}
    )
    data = response.json()["data"]["Infobase"]

    for item in data:
        assert filter_value.lower() in item["Location"].lower()



In [40]:
secure_query = """
query non_accessible {
  measures {
    measRepID
    mSRID
    sampleID
    purposeID
    polygonID
    siteID
    dataID
    measSetRepID
    aDateStart
    aDateEnd
    repDate
    specID
    fractionID
    measID
    groupID
    classID
    value
    unitID
    aggID
    index
    qualityFlag
    reportable
    lastEdited
    notes
  }
  samples {
    parSampID
    sampID
    mSRID
    contID
    siteID
    purposeID
    saMaterial
    dataID
    origin
    repType
    collType
    collPer
    collNum
    pooled
    collDT
    collDTStart
    collDTEnd
    sentDate
    recDate
    qualityFlag
    lastEdited
    notes
  }
}
"""


In [41]:
def is_empty_list(value):
    return isinstance(value, list) and len(value) == 0

def test_graphql_data_public_yield_empty_data():
    response = requests.post(
        "https://api-ipa.hc-sc.gc.ca/wastewater/", json={"query": secure_query}
    )
    data = response.json()["data"]

    for key, value in data.items():
        assert is_empty_list(value), f"'{key}' is not an empty list."

def test_graphql_data_group_yields_data():
    headers = {"X-Auth-Group": "hnj"}
    response = requests.post(
        "http://localhost:7071/graphql", json={"query": secure_query}, headers=headers
    )
    data = response.json()["data"]

    for key, value in data.items():
        assert contains_data(value), f"'{key}' is empty."

In [43]:
test_graphql_data_public_yield_empty_data()
test_graphql_data_group_yields_data()
test_graphql_public_tables()
test_graphql_filter()